In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
house_df = pd.read_csv('../Data/1976-2022-house.csv')
print(house_df.shape)
house_df = house_df[['year', 'state', 'state_po', 'party', 'candidatevotes', 'totalvotes', 'unofficial', 'district']]
house_df.head()

(32452, 20)


,year,state,state_po,party,candidatevotes,totalvotes,unofficial,district
0,1976,ALABAMA,AL,DEMOCRAT,58906,157170,False,1
1,1976,ALABAMA,AL,REPUBLICAN,98257,157170,False,1
2,1976,ALABAMA,AL,NaN,7,157170,False,1
3,1976,ALABAMA,AL,DEMOCRAT,66288,156362,False,2
4,1976,ALABAMA,AL,REPUBLICAN,90069,156362,False,2


In [4]:
# Won't count votes if the party is NaN, so replace with OTHER
house_df['party'].fillna('OTHER', inplace=True)
house_df.head()

,year,state,state_po,party,candidatevotes,totalvotes,unofficial,district
0,1976,ALABAMA,AL,DEMOCRAT,58906,157170,False,1
1,1976,ALABAMA,AL,REPUBLICAN,98257,157170,False,1
2,1976,ALABAMA,AL,OTHER,7,157170,False,1
3,1976,ALABAMA,AL,DEMOCRAT,66288,156362,False,2
4,1976,ALABAMA,AL,REPUBLICAN,90069,156362,False,2


In [5]:
house_df_2020_on = house_df.query('year >= 2020 and unofficial == False')
print(house_df_2020_on.shape)
house_df_2020_on.head()

(2816, 8)


,year,state,state_po,party,candidatevotes,totalvotes,unofficial,district
29636,2020,ALABAMA,AL,DEMOCRAT,116949,329075,False,1
29637,2020,ALABAMA,AL,REPUBLICAN,211825,329075,False,1
29638,2020,ALABAMA,AL,OTHER,301,329075,False,1
29639,2020,ALABAMA,AL,REPUBLICAN,197996,303569,False,2
29640,2020,ALABAMA,AL,DEMOCRAT,105286,303569,False,2


In [9]:
# We don't care about the districts so sum the votes for a party across all districts
votes_by_year_state_party = house_df_2020_on.groupby(['year', 'state_po', 'party'])['candidatevotes'].sum()
votes_by_year_state_party

year  state_po  party       
2020  AK        DEMOCRAT        159856
                OTHER             1183
                REPUBLICAN      192126
      AL        DEMOCRAT        608809
                OTHER            26838
                                 ...  
2022  WY        CONSTITUTION      4505
                DEMOCRAT         47250
                LIBERTARIAN       5420
                OTHER             8817
                REPUBLICAN      132206
Name: candidatevotes, Length: 469, dtype: int64

In [10]:
years = [2020, 2022]
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 
        'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 
        'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 
        'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
main_parties = ['DEMOCRAT', 'REPUBLICAN']

In [11]:
# make a dictionary where the first key:value is year:rest
# next is state:rest
# final is one of three parties (Democrat, Republican, Other) and their vote counts
# votes => first is year:states => states:parties => parties:votes
# votes[year][state][party] = votes
votes = {}
for year in years:
    votes[year] = {}
    for state in states:
        votes[year][state] = {}
        parties = list(votes_by_year_state_party[year][state].keys())
        for party in parties:
            if party in main_parties:
                votes[year][state][party] = votes_by_year_state_party[year][state][party]
            else:
                try:
                    votes[year][state]['OTHER'] += votes_by_year_state_party[year][state][party]
                except:
                    votes[year][state]['OTHER'] = votes_by_year_state_party[year][state][party] 

print(votes[2020])

{'AL': {'DEMOCRAT': 608809, 'OTHER': 26838, 'REPUBLICAN': 1416012}, 'AK': {'DEMOCRAT': 159856, 'OTHER': 1183, 'REPUBLICAN': 192126}, 'AZ': {'DEMOCRAT': 1629318, 'REPUBLICAN': 1638516, 'OTHER': 415}, 'AR': {'DEMOCRAT': 330485, 'OTHER': 20645, 'REPUBLICAN': 828266}, 'CA': {'DEMOCRAT': 11084234, 'REPUBLICAN': 5640667}, 'CO': {'OTHER': 107650, 'DEMOCRAT': 1679052, 'REPUBLICAN': 1378248}, 'CT': {'DEMOCRAT': 1022792, 'OTHER': 73485, 'REPUBLICAN': 676650}, 'DE': {'DEMOCRAT': 281382, 'OTHER': 10496, 'REPUBLICAN': 196392}, 'FL': {'DEMOCRAT': 4942287, 'OTHER': 53340, 'REPUBLICAN': 5469163}, 'GA': {'DEMOCRAT': 2393089, 'REPUBLICAN': 2490396, 'OTHER': 126}, 'HI': {'OTHER': 69807, 'DEMOCRAT': 354762, 'REPUBLICAN': 155215}, 'ID': {'OTHER': 32973, 'DEMOCRAT': 255531, 'REPUBLICAN': 561405}, 'IL': {'DEMOCRAT': 3355487, 'OTHER': 104403, 'REPUBLICAN': 2416929}, 'IN': {'DEMOCRAT': 1194901, 'OTHER': 62798, 'REPUBLICAN': 1738745}, 'IA': {'DEMOCRAT': 762271, 'OTHER': 78579, 'REPUBLICAN': 859418}, 'KS': {'DEM

In [12]:
print(votes[2020]['AL'])
# print(len(house_df_2020_on.query('year == 2020 and state_po == "AL"')))
# house_df_2020_on.query('year == 2020 and state_po == "AL"')

{'DEMOCRAT': 608809, 'OTHER': 26838, 'REPUBLICAN': 1416012}


# 2020 Census

In [13]:
census_df = pd.read_csv('../Data/2020-Census.csv')
census_df["POPULATION"] = census_df["POPULATION"].str.replace(',', '').astype(int)
census_df.head()

,AREA,CODE,POPULATION
0,Alabama,AL,5024279
1,Alaska,AK,733391
2,Arizona,AZ,7151502
3,Arkansas,AR,3011524
4,California,CA,39538223


In [33]:
census_df.set_index(census_df['CODE'], inplace=True)
census_df.loc['AL']['POPULATION']

5024279

# COVID-19 Data

In [15]:
covid_df = pd.read_csv('../Data/COVID-19_Reported.csv')
print(covid_df.shape)
covid_df['total_previous_day_cases'] = covid_df['previous_day_admission_adult_covid_confirmed'].fillna(0) + covid_df['previous_day_admission_pediatric_covid_confirmed'].fillna(0)
covid_df = covid_df[['date', 'state', 'total_previous_day_cases']]
covid_df.head()
# covid_df = covid_df[['state', 'date']]

(79823, 135)


,date,state,total_previous_day_cases
0,6/2/2021,PR,3.0
1,6/1/2021,ME,11.0
2,5/30/2021,MT,13.0
3,5/29/2021,AK,0.0
4,5/29/2021,MA,33.0


In [16]:
covid_df['month'] = covid_df['date'].str.split('/').str[0]
covid_df['year'] = covid_df['date'].str.split('/').str[2]

In [17]:
# Deaths by covid-19
# deaths_by_year_month_state = covid_df.groupby(['year', 'month', 'state'])['deaths_covid'].sum()
# print(deaths_by_year_month_state['2020']['3'].keys())

In [24]:
# # Checking if all of the states are in each month
# states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 
#         'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 
#         'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 
#         'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
# len_states = []
# for i in range(1, 13):
#     potential = list(cases_by_year_state['2020'][str(i)].keys())
#     flag = True
#     for s in states:
#         if s not in potential:
#             flag = False
#             print(s)
#             break
#     len_states.append(flag)

# # For the first two months of 2020, all of the states are not reporting data yet, 
# # so these months will be cut out, the rest are fine
# len_states

In [22]:
cases_by_year_state = covid_df.groupby(['year', 'state'])['total_previous_day_cases'].sum()
cases_by_year_state['2020'].keys()

Index(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI',
       'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN',
       'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH',
       'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI',
       'VT', 'WA', 'WI', 'WV', 'WY'],
      dtype='object', name='state')